# Modifiação do Dataset e envio para o RDS através do S3

In [ ]:
# Importação das bibliotecas necessárias
import boto3
import pandas as pd
from io import BytesIO
from sqlalchemy import create_engine
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from io import BytesIO
import os
from dotenv import load_dotenv

# Carrega variáveis de ambiente
load_dotenv()

In [ ]:
# Configuração da sessão e cliente S3 da AWS
boto3_session = boto3.Session()
s3_client = boto3_session.client('s3')

# Variáveis de ambiente para conexão com o banco de dados
database_name = os.getenv("DATABASE_NAME")
rds_host = os.getenv("RDS_HOST")
rds_port = os.getenv("RDS_PORT")
rds_user = os.getenv("RDS_USER")
rds_password = os.getenv("RDS_PASSWORD")

# Dowload do Dataset do S3

In [ ]:
# Faz o download do arquivo do S3
bucket_name = os.getenv("BUCKET_NAME")
file_name = 'dataset/Hotel Reservations.csv'
response = s3_client.get_object(Bucket=bucket_name, Key=file_name)
data = response['Body'].read()

    # Carrega os dados do arquivo CSV
hotel_reservations_modified = pd.read_csv(BytesIO(data))


# Modificação do dataset

In [ ]:
# Adiciona a coluna label_avg_price_per_room e remove a coluna avg_price_per_room do dataset
def label_avg_price(price):
    if price <= 85:
        return 1
    elif price < 115:
        return 2
    else:
        return 3

hotel_reservations_modified['label_avg_price_per_room'] = hotel_reservations_modified['avg_price_per_room'].apply(label_avg_price)
hotel_reservations_modified = hotel_reservations_modified.drop('avg_price_per_room', axis=1)

print("Coluna 'label_avg_price_per_room' adicionada e coluna 'avg_price_per_room'.")

# Envio do dataset modificado para o S3

In [ ]:
# Transforma a variável hotel_reservatins_modified em um arquivo csv
hotel_reservations_modified.to_csv("hotel Reservations Modified.csv", index = False)
print("Variável 'hotel_reservations_modified' transformada e salva como arquivo CSV")

In [ ]:
try:
    # Salva no modificado no S3
    s3_client.upload_file("Hotel Reservations Modified.csv", bucket_name, "dataset/Hotel Reservations Modified.csv")
    print("Arquivo enviado para o S3")
except Exception as e:
     print(f"Erro ao enviar arquivo para oS3: {e}")


# Envio dos datasets do S3 para o RDS

In [ ]:
# Carrega arquivos CSV do S3 em um DataFrame do Pandas
def load_csv_data(s3_bucket):
    paginator = s3_client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=s3_bucket)
    
    dfs = []
    
    for page in page_iterator:
        if 'Contents' in page:
            for file_object in page['Contents']:
                file_key = file_object['Key']
                
                if file_key.endswith('.csv'):  # Verifica se o arquivo tem extensão .csv
                    try:
                        file_obj = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
                        df = pd.read_csv(BytesIO(file_obj['Body'].read()))
                        dfs.append(df)
                    except Exception as e:
                        print(f"Erro ao ler o arquivo {file_key}: {e}")
    
    if dfs:
        return dfs  # Retorna um único DataFrame combinando todos os CSVs
    else:
        return pd.DataFrame()  # Retorna um DataFrame vazio se nenhum CSV for encontrado


In [ ]:
# Carregar dados CSV do S3 em um DataFrame do Pandas
try:
    df = load_csv_data(bucket_name)
    print("Dados carregados")
except Exception as e:
    print(f"Erro ao carregar dados: {e}")

In [ ]:
try:
    # Conecta ao RDS
    conn_str = f'mysql+pymysql://{rds_user}:{rds_password}@{rds_host}:{rds_port}/{database_name}'
    engine = create_engine(conn_str)

    # Escreve o DataFram no RDS
    df[0].to_sql("Hotel Reservation Modified", con=engine, if_exists='replace', index=False)
    df[1].to_sql("Hotel Reservation", con=engine, if_exists='replace', index=False)
    
    engine.dispose()

    print('Arquivos enviados para o Rds')
    
except Exception as e:
    print(f"Ocorreu um erro ao enviar arquivos para o RDS: {e}")